# Recommender with Spotify API Integration

## 1: Create Spotify Authorization Token

Due to Spotify API restrictions, you need to add SpotifyForDevelopers to your Spotify account to be able to run the code in this notebook. Since you can't "log in" per se from the notebook itself, creating this token allows you to validate your credentials and retrieve information from your Spotify account. 

### Steps

1. Go to https://developer.spotify.com/dashboard/ and log in (or sign up if you don't already have an account).  
2. Go to https://developer.spotify.com/console/post-playlists/ and press "Get Token".
3. Select the following checkboxes:  
    a. playlist-modify-public  
    b. playlist-modify-private  
    c. user-read-recently-played  
4. Press "Request Token" and "Agree". Now you should see aa field value under **OAuth Token**. Copy this, as we'll need it later.  
5. Next, open Spotify. Open the drop-down next to your name in the top-right corner, and select **Account**.  
6. Copy the **Username** as we'll need it later.  
7. Open a terminal on your system. 
8. Enter the 2 following commands one by one in your terminal, replacing the <> with the variables you copied earlier. This basically allows you to validate your Spotify credentials without hard-coding it in this notebook's code cells. If you're on a Mac, replace the _SET_ keyword with _export_.  
**setx SPOTIFY_AUTHORIZATION_TOKEN \<yourOAuthToken\>  
setx SPOTIFY_USER_ID \<yourUsername\>**  

<img src="./terminal.JPG">

Restart your system for the changes to take effect. 

**Note: Spotify Access Tokens expire every hour, so you may need to go through the previous steps again if any of the code cells fail.**

In [1]:
#!pip install spotipy
import spotipy
import json
import os
import requests

# instantiate global variables
auth_token = os.environ.get("SPOTIFY_AUTHORIZATION_TOKEN") 
user_id = os.getenv("SPOTIFY_USER_ID")

# if the below lines returns None, please restart your system
# print(auth_token)
# print(user_id)

In [4]:
def getAPIrequest(auth_token, url):
    """
    Function to place GET requests to the Spotify API.
    """
    response = requests.get(
            url,
            headers={
                "Content-Type": "application/json",
                "Authorization": f"Bearer {auth_token}"
            }
        )
    return response

def getLastPlayedSongs(numOfTracks):
    """
    Function to retrieve the last numOfTracks songs played by the user.
    """
    url = f"https://api.spotify.com/v1/me/player/recently-played?limit={numOfTracks}"
    response = getAPIrequest(auth_token, url)
    response_json = response.json()
    songs = []
    print(json.dumps(response_json, indent=4))
    for song in response_json["items"]:
        songs.append(song)
    print(songs)
    #return songs


{
    "items": [
        {
            "track": {
                "album": {
                    "album_type": "album",
                    "artists": [
                        {
                            "external_urls": {
                                "spotify": "https://open.spotify.com/artist/20wkVLutqVOYrc0kxFs7rA"
                            },
                            "href": "https://api.spotify.com/v1/artists/20wkVLutqVOYrc0kxFs7rA",
                            "id": "20wkVLutqVOYrc0kxFs7rA",
                            "name": "Daniel Caesar",
                            "type": "artist",
                            "uri": "spotify:artist:20wkVLutqVOYrc0kxFs7rA"
                        }
                    ],
                    "available_markets": [
                        "AD",
                        "AE",
                        "AG",
                        "AM",
                        "AO",
                        "AR",
                        "AT",
          

[{'track': {'album': {'album_type': 'album',
    'artists': [{'external_urls': {'spotify': 'https://open.spotify.com/artist/20wkVLutqVOYrc0kxFs7rA'},
      'href': 'https://api.spotify.com/v1/artists/20wkVLutqVOYrc0kxFs7rA',
      'id': '20wkVLutqVOYrc0kxFs7rA',
      'name': 'Daniel Caesar',
      'type': 'artist',
      'uri': 'spotify:artist:20wkVLutqVOYrc0kxFs7rA'}],
    'available_markets': ['AD',
     'AE',
     'AG',
     'AM',
     'AO',
     'AR',
     'AT',
     'AU',
     'AZ',
     'BB',
     'BD',
     'BE',
     'BF',
     'BG',
     'BH',
     'BI',
     'BJ',
     'BN',
     'BO',
     'BR',
     'BS',
     'BT',
     'BW',
     'BZ',
     'CA',
     'CH',
     'CL',
     'CM',
     'CO',
     'CR',
     'CV',
     'CW',
     'CY',
     'CZ',
     'DE',
     'DJ',
     'DK',
     'DM',
     'DO',
     'DZ',
     'EC',
     'EE',
     'EG',
     'ES',
     'FI',
     'FJ',
     'FM',
     'FR',
     'GA',
     'GB',
     'GD',
     'GE',
     'GH',
     'GM',
     'GN',


## 2: Retrieve User's Recently Played Songs
https://developer.spotify.com/console/get-recently-played/
 
JSON format is specified here. 

## 3: Get User's Preferences

## 4: Convert to Model-friendly Input

## 5: Make Recommendations